$$\newcommand{\fudm}[2]{\frac{\mathrm{D} #1}{\mathrm{D} #2}}
\newcommand{\pad}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\ppad}[2]{\frac{\partial^2 #1}{\partial #2^2}}
\newcommand{\ppadd}[3]{\frac{\partial^2 #1}{\partial #2 \partial #3}}
\newcommand{\nnabla}{\nabla^2}
\newcommand{\eps}{\epsilon}
\newcommand{\vdetail}[1]{\vb{#1}=\begin{pmatrix}#1_1\\#1_2\\#1_3\end{pmatrix}}
\newcommand{\vb}[1]{\mathbf{#1}}
\newcommand{\va}[1]{\vec{#1}}
\newcommand{\tb}[1]{\underline{\underline{\mathbf{#1}}}}
\newcommand{\fud}[2]{\frac{\mathrm{d} #1}{\mathrm{d} #2}}$$

# Spherical Waves

In spherical coordinates $(r,\phi,\theta)$ with 

\begin{eqnarray}
x&=&r \sin \theta \cos \phi\\
y&=&r \sin \theta \sin \phi\\
z&=&r \cos \theta
\end{eqnarray}

the Laplacian operator is

\begin{equation}
\nabla^2=\ppad{}{r} + \frac{2}{r}\pad{}{r} +\frac{1}{r^2 \sin \theta}\pad{}{\theta}\left(\sin \theta \pad{}{\theta}\right) +
\frac{1}{r^2 \sin^2 \theta} \ppad{}{\phi} 
\end{equation}

If we assume spherical symmetry the the acoustic pressure is a function of $r$ only and the Laplacian simplifies to

\begin{equation}
\nabla^2=\ppad{}{r}+\frac{2}{r}\pad{}{r}\quad .
\end{equation}

Thus the wave equation becomes

\begin{equation}
\ppad{p}{r}+\frac{2}{r}\pad{p}{r}=\frac{1}{c^2}\ppad{p}{t}\quad .\tag{4.1}
\end{equation}

We can rewrite Eq. (4.1) as

\begin{equation}
\ppad{(p r)}{r}=\frac{1}{c^2}\ppad{(r p)}{t}\quad ,
\end{equation}

which resembles a plane wave equation for the product of the variable $p$ and $r$. Thus the solution to this equation is

\begin{equation}
p r = f_1(c t - r) + f_2 (c t +r)
\end{equation}

or 

\begin{equation}
p = \frac{1}{r} f_1(c t - r) + \frac{1}{r} f_2 (c t +r)\quad , \tag{4.2}
\end{equation}

for all $r>0$, the solution is not valid at $r=0$. Equation (4.2) is an outgoing and incoming wave where the amplitude is proportional to $1/r$. The incoming wave diverges at $r=0$. The reason why our equation becomes invalid is that the small amplitude limit is not valid anymore, nonlinearities build up and we need a better description which accounts for *finite amplitude effects*.




In [2]:
%pylab inline
import matplotlib.pyplot as plt #plotting
import numpy as np #array functions
import math as m
from IPython import display #for continous display
from ipywidgets import widgets #for the widgets
pylab.rcParams['figure.figsize'] = (12, 8)
pylab.rcParams['font.size'] = 15

nimg=0

def plotwave(u,time,r):
    plt.figure(1)
    plt.clf()
    plt.plot(r,u)
    plt.text(0.1,3.4,"time {0:.5f}".format(time)) #annotate the time
    plt.xlabel(r"r")
    plt.gca().set_ylim([-4,4])
    display.clear_output(wait=True)
    display.display(plt.gcf())
        
def solvewave(b):
    tabs.visible=False
    #computational domain
    nx = 381
    size=2. #size of the domain
    #parameters of the wave
    c = 5. #speed of sound
    l=w_wavelength.value #wavelength
    nu=c/l #frequency
    omega=nu*2.*m.pi #angular frequency
    duration=w_sourceduration.value/nu #duration of source
    
    #further variables
    dx = size/(nx-1)
    CFL=0.1 #CFL number <1
    dt = CFL*dx/c 
    nt=int(w_simduration.value/dt) #number of time steps
    if w_position.value=='Left':
        sourcepos=1
        ampl=100.
    else:
        sourcepos=int(nx/2)
        ampl=1.
    r=numpy.arange(dx,nx*dx,dx) #radius
    
    #every xx times over the total nt timesteps an output should be generated 
    output=map(int,list(numpy.linspace(1,nt,int(nt/50))))

    u  = numpy.zeros(nx) #pressure at t
    un = numpy.zeros(nx) #pressure at t-dt
    unn= numpy.zeros(nx) #pressure at t-2*dt
    C=c*c*dt*dt/dx/dx
    
    #Assign initial conditions, for a sin^2 wave
    if w_source_type.value=="Time Dependent":
        un[sourcepos]=0.    #amplitude is sin(omega*t) with t=0
        unn[sourcepos]=1.*r[sourcepos]   #velocity  is cos(omega*t) with t=0
    else: #or set it as an initial value
        for xx in range(nx):
            x=float(xx)*dx-size/2. #0 at the center
            unn[xx]=(m.cos(2.*m.pi/l*x)**2)*float(abs(x)<(c*duration/2.))
        for xx in range(1,nx-1): #calculate t=0 time step
            un[xx] = unn[xx] - 0.5*C*(unn[xx+1]-2.*unn[xx]+unn[xx-1])
    
    #plt.figure(1, figsize=(8, 4), dpi=300)

    for n in range(nt+1): ##loop across number of time steps
        #this line computes the finite differences of the wave equation
        u[1:-1]=2.*un[1:-1]-unn[1:-1]+C*(un[:-2]+un[2:]-2.*un[1:-1])
 
        #Boundary conditions right
        if w_boundary_r.value=='Open':
            u[-1] = un[-1]-dt*c/dx*(un[-1]-un[-2]) 
        else:
            u[-1] = u[-2]
        #Boundary conditions right
        u[0] = u[1]
            
        #pressure source
 
        if w_source_type.value=="Time Dependent":
            if float(n*dt<duration):
                u[sourcepos]=ampl*m.sin(omega*n*dt)**2*float(n*dt<duration)*r[sourcepos]
                
        #save values for the time derivative 
        unn=un.copy() #n-1 time step
        un=u.copy()   #n time step
        
        if (n in output):
            plotwave(u[1:]/r,n*dt,r)
            
    #and plot the last figure    
    #plotwave(u[1:]/r,n*dt,r)
    tabs.visible=True
    display.display(tabs)

    display.display(w_start) 

#setup the graphical interface 
w_wavelength=widgets.FloatSlider(description="Wavelength",value=.5,min=0.1,max=1)
w_sourceduration=widgets.FloatSlider(description="Emission Duration in Periods",value=.5,min=0.5,max=10)
w_simduration=widgets.FloatSlider(description="Simulation Duration",value=.2,min=0.1,max=5)
w_position=widgets.RadioButtons(description="Source Position",options=["Left", "Center"],value="Center")
w_start=widgets.Button(description="Start Simulation")
w_start.on_click(solvewave)
w_boundary_r=widgets.RadioButtons(description="Right Boundary",options=["Reflective", "Open"],value="Reflective")
w_source_type=widgets.RadioButtons(description="Type of Source",options=["Time Dependent", "Inital Condition"],value="Time Dependent")

page1=widgets.Box([w_wavelength,w_sourceduration,w_position,w_simduration])
page2=widgets.Box([w_source_type])
page3=widgets.Box([w_boundary_r])
tabs = widgets.Tab(children=[page1, page2, page3])
tabs.set_title(0, 'Wave')
tabs.set_title(1, 'Source')
tabs.set_title(2, 'Boundaries')
display.display(tabs)

display.display(w_start)    

Populating the interactive namespace from numpy and matplotlib


/Users/cdohl/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['display']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


VGFiKGNoaWxkcmVuPShCb3goY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuNSwgZGVzY3JpcHRpb249dSdXYXZlbGVuZ3RoJywgbWF4PTEuMCwgbWluPTAuMSksIEZsb2F0U2xpZGVyKHbigKY=


Button(description=u'Start Simulation', style=ButtonStyle())

In [5]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom2.css", "r").read()
    return HTML(styles)
css_styling()